# Output PDBs with teh *average* escape scores for a given antibody class or plasma group as B factors
This Python Jupyter notebook outputs PDBs with the escape scores as B factors.

Though we will want more elaborate series of commands to codify our visualization of these RBD structures colored by escape, the series of commands below, when executed in a `PyMol` session with one of these PDBs open, will color the RBD surface according to escape scores.

We want to take 

Alternatively, it might be worth it to take a weighted average of the escape fractions for each condition within in a given group. This might be important because some (noisy) sera have relatively high escape fractions across the entire RBD. We currently show these as flat escape profiles in the logo plots, with ylims that are calculated according to specs in the `escape_profiles_config.yaml` file. These ylims are saved to an output CSV file in the `results/escape_profiles/` subdirectory, so it would be possible to normalize to that y-max value before averaging. This may not be necessary, so we should see how it looks first. 

Also, we are only interested in the toal escape at a site, not the max mutation effect. 
     
We write PDBs with B factors indicating the group-average total site escape.

First, import Python modules:

In [1]:
import collections
import copy
import os
import warnings

import Bio.PDB

import dms_variants.pdb_utils

from IPython.display import display, HTML

import pandas as pd

import yaml

Read the configuration file:

In [2]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

Read escape profiles config file and configuration for outputting PDBs:

In [3]:
print(f"Reading escape profiles configuration from {config['escape_profiles_config']}")
with open(config['escape_profiles_config']) as f:
    escape_profiles_config = yaml.safe_load(f)

print(f"Reading PDB output configuration from {config['average_output_pdbs_config']}")
with open(config['average_output_pdbs_config']) as f:
    average_output_pdbs_config = yaml.safe_load(f)

Reading escape profiles configuration from data/escape_profiles_config.yaml
Reading PDB output configuration from data/average_output_pdbs_config.yaml


Make output directory:

In [4]:
os.makedirs(config['average_output_pdbs_results_dir'], exist_ok=True)

Read escape fractions and compute **total** and **maximum** escape at each site, and also the total and maximum escape at each site normalized to be between 0 and 1 for each selection:

In [5]:
print(f"Reading escape fractions from {config['escape_fracs']}")

escape_fracs = (
    pd.read_csv(config['escape_fracs'])
    .query('library == "average"')
    .assign(site=lambda x: x['label_site'])
    .groupby(['selection', 'site'])
    .aggregate(total_escape=pd.NamedAgg(config['mut_metric'], 'sum'),
#                max_escape=pd.NamedAgg(config['mut_metric'], 'max')
               )
    .reset_index()
    )

display(HTML(escape_fracs.head().to_html(index=False)))

Reading escape fractions from results/escape_scores/escape_fracs.csv


selection,site,total_escape
12C_d152_80,331,0.049257
12C_d152_80,332,0.109619
12C_d152_80,333,0.051312
12C_d152_80,334,0.153061
12C_d152_80,335,0.115742


Now map the escape metrics to the B-factors.
For sites where no mutations have escape scores:
 - In the RBD chain(s) fill the B-factor for non-normalized scores to -1 to enable collapsing to zero or callout as a a separate class, depending how we choose to color sites for different visualizations. 
 - In other chains, always fill missing B factors to 0.  

In [6]:
pdbfile = average_output_pdbs_config['pdb']['pdbfile']
assert os.path.isfile(pdbfile)

rbd_chains = average_output_pdbs_config['pdb']['chains']

for name, ab_class in average_output_pdbs_config['conditions'].items():
    print(f"\nMaking PDB mappings for {name} to {pdbfile}")
    
    # get conditions from escape_profiles_config.yaml
    assert isinstance(ab_class, list)
    conditions = []
    for group in ab_class:
        assert group in escape_profiles_config
        conditions += list(escape_profiles_config[group]['conditions'])
    print(f"Making mappings for {len(conditions)} conditions.")
    
    # get escape fracs just for conditions of interest
    df = escape_fracs.query('selection in @conditions')
    
    # assign average total_escape at each site across all the conditions in ab_class
    df = (df
          .assign(ab_class=name)
          .groupby(['site'])
          .aggregate(mean_total_escape=pd.NamedAgg('total_escape', 'mean'),
                      )
          .reset_index()
          .drop_duplicates()
         )
    
    # get chains
    assert isinstance(rbd_chains, list)
    print('Mapping to the following chains: ' + ', '.join(rbd_chains))
    df = pd.concat([df.assign(chain=chain) for chain in rbd_chains], ignore_index=True)
    
    
    # make mappings for each condition and metric
    print(f"  Writing B-factor re-assigned PDBs for {name} to:")

    for metric in ['mean_total_escape']:

        # what do we assign to missing sites?
        missing_metric = collections.defaultdict(lambda: 0)  # non-RBD chains always fill to zero
        for chain in rbd_chains:
            if 'norm' in metric:
                missing_metric[chain] = 0  # missing sites in RBD are 0 for normalized metric PDBs
            else:
                missing_metric[chain] = -1  # missing sites in RBD are -1 for non-normalized metric PDBs

        fname = os.path.join(config['average_output_pdbs_results_dir'], f"{name}_6m0j_{metric}.pdb")
        print(f"    {fname}")

        dms_variants.pdb_utils.reassign_b_factor(input_pdbfile=pdbfile,
                                                 output_pdbfile=fname,
                                                 df=df,
                                                 metric_col=metric,
                                                 missing_metric=missing_metric)


Making PDB mappings for class1 to data/pdbs/6M0J.pdb
Making mappings for 6 conditions.
Mapping to the following chains: E
  Writing B-factor re-assigned PDBs for class1 to:
    results/bjorkman_custom_analyses/pdbs/class1_6m0j_mean_total_escape.pdb

Making PDB mappings for class2 to data/pdbs/6M0J.pdb
Making mappings for 6 conditions.
Mapping to the following chains: E
  Writing B-factor re-assigned PDBs for class2 to:
    results/bjorkman_custom_analyses/pdbs/class2_6m0j_mean_total_escape.pdb

Making PDB mappings for class3 to data/pdbs/6M0J.pdb
Making mappings for 5 conditions.
Mapping to the following chains: E
  Writing B-factor re-assigned PDBs for class3 to:
    results/bjorkman_custom_analyses/pdbs/class3_6m0j_mean_total_escape.pdb

Making PDB mappings for class4 to data/pdbs/6M0J.pdb
Making mappings for 4 conditions.
Mapping to the following chains: E
  Writing B-factor re-assigned PDBs for class4 to:
    results/bjorkman_custom_analyses/pdbs/class4_6m0j_mean_total_escape.pdb
